In [ ]:
import numpy as np 
import pandas as pd

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
# plotly.init_notebook_mode()
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True)  
import plotly.express as px
import os



# Lire et exploration des données 




## Affichage des données




In [ ]:
with open('busy_day.in') as file:
    data_list = file.read().splitlines()

print('  ----  Information general ---- '
      '\n Ligne de la grille: ',data_list[0].split(" ")[0], 
      '\n Colonnes de la grille: ', data_list[0].split(" ")[1], 
      '\n Le nombre de drones: ', data_list[0].split(" ")[2], 
      '\n Le nombre de tours: ', data_list[0].split(" ")[3],
      '\n La capacité maximum d\'une drone en (u): ', data_list[0].split(" ")[4],
      '\n\n   ----Information Produit ----  ',
#         '\n Le nombre total des produits disponibles: ', products_df['sum'].sum(),

      '\n Different types de produit: ',data_list[1],
      '\n\n   ---- Information entrepots----  ',
      '\n Nombre d\'entrepots: ',data_list[3],
#        '\n Les produits dans le 1 entrepot: ', data_list[5],
      '\n\n   ---- Information Ordres ----  ',
      '\n  Nombre d\'ordres: ', data_list[24])
    

## Information sur les entrepots
Lecture des informations des entrepots et structurer les données en dataframe: 
* row, column: la localisation d'entrepot sur la grille. 
* id : l'id de chaque entrepot. 

In [ ]:
# Défnition de la localisation de chaque entrepot
wh_locations = data_list[4:24:2]
wh_id_list = list(range(len(wh_locations)))
wh_rows_list = [ware_house_r.split()[0] for ware_house_r in wh_locations]
wh_cols_list = [ware_house_c.split()[1] for ware_house_c in wh_locations]

# Creation du dataframe
warehouse_df = pd.DataFrame({'id_entrepot': wh_id_list, 'row': wh_rows_list, 'col': wh_cols_list})
warehouse_df = warehouse_df.astype(int)
# print('Localilatio')
warehouse_df


fig = px.scatter(warehouse_df, x="row", y="col",    color_continuous_scale='greens')

iplot(fig)

## Information sur les produits
Lecture des informations des produits et structurer les données en dataframe, et assigner chacun de produits aux entrepots.

In [ ]:
# Structurer tous les produits liés à chaque entrepôt
cols = [f'warehouse_{i}' for i in range(len(warehouse_df))]
# Obtenir tout le contenu de chaque entrepôt
products_df = pd.DataFrame([x.split() for x in data_list[5:24:2]]).T
products_df.columns = cols # renomer les colonnes

products_df

In [ ]:
# Attaché à chaque type de produit son poids
products_df['weight'] = data_list[2].split()

# Attaché à chaque type de produit son id
products_df['id'] = list(range(len(products_df)))

# Réorganiser les colonnes du dataframe
cols = products_df.columns.to_list()
cols.remove("id")
products_df = products_df[["id"] + cols].astype(int)

products_df

Distribution des produits dans les entrepots. 

In [ ]:
products_df.sum()

## Les informations sur les ordres

In [ ]:
# Obtenez la plus grandes commande en termes de produits
max_order = max([len(x.split()) for x in data_list[27:3775:3]])
print(f"La plus grande commande a  {max_order} types de produits!")

# Créer le numéro de produit de la commande
cols_order = [f'item_{i}' for i in range(max_order)]

# Creation des ordres dataframes
order_df = pd.DataFrame([x.split() for x in data_list[27:3775:3]]).fillna(0)
order_df.columns = cols_order

#  the ordre items 
order_df['order_items'] = data_list[26:3775:3]

# Définir la position commande dans la grille
order_df['coor_x'] = [x.split()[0] for x in data_list[25:3775:3]]
order_df['coor_y'] = [x.split()[1] for x in data_list[25:3775:3]]

# Garantir que tous les colonnes sont des nombres 
order_df = order_df.astype(int)

order_df

# Analyse les données structurées 


In [ ]:
# Calcul du nombre de produits dans chaque entrepôt
warehouse_df["prod_count"] = products_df.sum().iloc[1:-1].to_list()
warehouse_df

In [ ]:
# Calcul du poids de l'entrepôt en produits
warehouse_weight_list = []
weights = products_df["weight"].to_list()
for col in products_df.columns.to_list()[1:-1]:
    pwh_count = products_df[col].to_list()
    warehouse_weight_list.append(
        sum([q * w for q, w in zip(pwh_count, weights)])
    )

warehouse_df["total_weight"] = warehouse_weight_list

warehouse_df.head(10)

In [ ]:
# Calcul de la densité de l'entrepôt
warehouse_df["density"] = warehouse_df["total_weight"] / warehouse_df["prod_count"] 
warehouse_df["density"]  = warehouse_df["density"].round(0)
warehouse_df.head(10)


In [ ]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import plotly.express as px

# fig = px.scatter(order_df, x="row", y="col", text="order_items",color="order_items", color_continuous_scale='icefire')

iplot(fig)

# Construire le simulateur de drone


In [ ]:
warehouse_df.head()

In [ ]:
products_df.head()

In [ ]:
order_df.head()

## Initialisation des données pour le simulateur

In [ ]:
# le nombre de drones
num_drones = int(data_list[0].split(" ")[2])

# le nombre des orders
num_orders = len(order_df)

# le nombre des turns
num_turns = int(data_list[0].split(" ")[3])

# Nombre maximum de livraisons par drone
max_deliveries = int(num_orders // num_drones)

print(f"Le nombre maximum de livraisons par drone: {max_deliveries}")

In [ ]:
num_turns


## Drones class

In [ ]:
import sys
import math
import pickle

class drone:
    """
        la class drone simule le movement du drone des entreport vers les clients et des clients vers les entrepots. 
        Le movement de drone en principe c'est simple: 

        La drone recois une commande pour la livraison et chaque drone est responsable d'une liste de livraison.
        Elle livre chaque type de produit à un client

        Le processus va etre optimiser par la suite.  
    """
    
    def __init__(self, did=None, delivery_order=None, verbose=False):
        """
            Intialisation de la drone avec un contenu particulier pour la simulation.
            
            The `delivery_order` parameter:  Porte les informations des commandes que ce drone particulier doit livrer.
            Notez que les bons de livraison sont fournis comme suit :
            `[ (order_id, [item_1, ..., item_n], [delivery_x, delivery_y]), ... ]`.
        """
        
        #  Identification de Drone
        self.drone_id = did
        self.verbose = verbose
        
        # Status de Drone
        self.transporting = False
        self.future_position = [0, 0]
        self.at_order = None
        self.at_product = None
        
        ## Information du voyage
        #  Informations des Ordres
        self.delivery_target = None
        
        #  Informations du produits
        self.distance_to_delivery = None
        self.carring_product = None
        
        # Les informations internes du drone
        self._deliveries = delivery_order # [ ("order", "products", "position") ]
        self._delivered_orders = 0
        self._distance = 0
        

    def start(self, wh_logger=None):
        """
            Cette méthode est chargée de démarrer le drone, et de donner la première tâche pour chaque existant
            Drone. Notez que nous allons d'abord restructurer la liste de livraison en une liste de produits.
            Le drone livrera des produits à chaque emplacement client. Cette fonction reçoit le
            Paramètre `wh_logger` qui est l'information de journalisation des entrepôts et des produits
            Disponible dans chaque entrepôt. Ceci est nécessaire car le drone recherchera toujours le
            Produit disponible le plus proche, puis ce drone retirera ce produit de l'entrepôt
            Par conséquent, il supprimera le produit des journaux de l'entrepôt.

            
            :param tuple wh_logger: Un tuple avec le dataframe des produits et des entrepôts, respectivement.
            
            :returns: L'enregistreur modifié des produits et des entrepôts
            :rtype: tuple
        """
        
        #  Réorganisation des Livraisons 
        new_struc_delivery = []
        for delivery in self._deliveries:
            for del_item in delivery[1]:
                new_struc_delivery.append( (delivery[0], del_item, delivery[2]) )
        self._deliveries = new_struc_delivery

        # Créer le produit et la commande actuels
        self.at_order = self._deliveries[0][0]
        self.at_product = self._deliveries[0][1]
        
        if self.verbose:
            self.summary()

        # Calculer la distance au produit suivant
        self.distance_to_delivery, self.future_position, logger = self.find_next_product(wh_logger)

        # Définir l’objectif de livraison à l’entrepôt
        self.transporting = True
        self.delivery_target = "warehouse"
        self.at_product = None

        return logger
    
    
    def step(self, wh_logger=None):
        """
            Cette fonction fera marcher le drone une fois vers le futur. 
            Si le drone transporte un produit ou se rend en entrepôt pour obtenir un produit, 
            la méthode de l’étape fera en sorte que le drone se rapprochera un peu plus de sa cible.
            Si le drone est au client ou à l’entrepôt, la méthode de l’étape livrera le produit au client,
            ou il obtiendra le produit de l’entrepôt, en particulier. Ensuite, il guidera le drone vers
            l’entrepôt ou le client si le drone est dans le client ou l’entrepôt, respectivement.

            
            :param tuple wh_logger:Un tuple avec le dataframe produits et entrepôts, respectivement.
            
            :returns: L’enregistreur modifié des produits et des entrepôts.
            :rtype: tuple
        """
        
        # ACTION 1 = Déplacer le drone s’il n’est pas dans la cible
        if self.transporting:
            # Augmenter la distance 
            self._distance += 1
            # Si la position n’est pas la livraison
            if self.distance_to_delivery != 0:
                self.move_step() # Déplacez le drone d’une étape vers la cible
                return None
            elif self.distance_to_delivery == 0:
                return self.delivery_action(wh_logger) # Charger / Décharger le drone
            else: 
                print("Drone distance error!")
        
        
    def move_step(self):
        """
            Cette méthode est utilisée lorsque le drone transporte un produit.

            La méthode du pas de mouvement est responsable de rapprocher le drone d'un pas plus près de sa cible
            à chaque fois. 
        """
        
        if self.drone_id == 2 and self.verbose:
            print(f"Déplacer le drone vers {self.distance_to_delivery}")
        
        if self.distance_to_delivery != None:
            self.distance_to_delivery -= 1
    
    
    def delivery_action(self, wh_logger=None):
        """
            Cette méthode est utilisée lorsque le drone arrive chez le client ou l'entrepôt.

            La méthode d'action de livraison fera l'action de livraison à l'entrepôt ou chez le client, il
            dépend de l'endroit où se trouve le drone.

            Si le drone est chez le client, l'action de livraison livrera le produit au client, puis
            retirer un produit de la liste des produits que le drone doit livrer, puis guider le drone
            à l'entrepôt le plus proche qui a le prochain produit à livrer. Cela change aussi le
            journalisation de l'entrepôt, en supprimant l'élément que le drone obtiendra de la journalisation de l'entrepôt,
            ce qui rend le produit «reservation» pour ce drone particulier. Ensuite, il calcule la tragectoire à l'entrepôt
            et entrez dans le mode de transport pour vous rendre à l'entrepôt.

            Si le drone est dans l'entrepôt, le drone calcule la tragectoire au client et récupère le produit
            qu'il doit livrer au client en définissant l'attribut `at_product`. Puis il entre dans le transport
            mode pour aller au client.
            
            :param tuple wh_logger:Un tuple avec le dataframe produits et entrepôts, respectivement.
            
            :returns: L’enregistreur modifié des produits et des entrepôts.
            :rtype: tuple
        """
        
        if self.verbose:
            print(f"Drone {self.drone_id} - Lors de l'action de livraison...")
        
        if self.delivery_target == "warehouse":    
            
            if self.verbose:
                print(f"Drone {self.drone_id} - A l'entrepôt!")
            
            # Calculer la distance euclidienne au suivant 
            self.distance_to_delivery = ((self.future_position[0] - self._deliveries[0][2][0])**2 + (self.future_position[1] - self._deliveries[0][2][1])**2)**0.5
            self.distance_to_delivery = math.ceil(self.distance_to_delivery) # la valeur entiere 
            
            # Définir le produit actuel
            self.at_product = self._deliveries[0][1]
            
            # Définir la position future du drone
            self.future_position = self._deliveries[0][2]
            
            # Définir l'objectif de livraison au client
            self.delivery_target = "client"
            
            if self.verbose:
                print(f"Drone {self.drone_id} - va chez le client...")
            
            return None
            
        elif self.at_product != None and self.delivery_target == "client":
            
            if self.verbose:
                print(f"Drone {self.drone_id} - Chez le client !")
            
            # Vérifiez si le drone est terminé
            if len(self._deliveries) != 1: 
                # Supprimer la livraison actuelle
                self._deliveries = self._deliveries[1:]
                
                # Vérifiez si la commande est terminée
                if self.at_order != self._deliveries[0][0]:
                    self._delivered_orders += 1

                # Créer le produit actuel et commander
                self.at_order = self._deliveries[0][0]
                self.at_product = self._deliveries[0][1]
                
                if self.verbose:
                    self.summary()
                
                # Calculer la distance jusqu'au prochain produit
                self.distance_to_delivery, self.future_position, logger = self.find_next_product(wh_logger)
                
                # Définir l'objectif de livraison à l'entrepôt
                self.delivery_target = "warehouse"
                self.at_product = None
                
                return logger
            
            else:
                # Ne pas transporter
                self.transporting = False
                return wh_logger
            
        else: 
            if self.verbose:
                print("Drone delivery action error!")
                
    
    
    def find_next_product(self, wh_logger=None):
        """
            C'est la méthode qui est utilisée par l'action de livraison, lorsqu'elle est chez le client, pour trouver
            l'entrepôt avec Le produit souhaité le plus proche. Cette méthode examine toutes les informations 
            de journalisation des entrepôts pour rechercher Le prochain produit qui sera livré et calcule la distance
            jusqu'à cet entrepôt. Après elle sélectionne L'entrepôt le plus proche pour que le drone aille chercher le produit.
            
            
            :param tuple wh_logger:Un tuple avec le dataframe produits et entrepôts, respectivement.
            
            :returns: L’enregistreur modifié des produits et des entrepôts.
            :rtype: tuple
        """
        
        # Rupture du contenu de l’enregistreur d’événements
        product_df = wh_logger[0].copy()
        warehouse_df = wh_logger[1].copy()
        
        # Calculer la distance à l’entrepôt le plus proche avec le produit
        prod_quant = product_df.iloc[self.at_product][1:-1].to_list()
        
        # Initialiser la liste utilisée
        distance, warehouses = [], []
        for wid, quantity in enumerate(prod_quant):
            #Vérifier la quantité de produit
            if quantity != 0:
                # Inclure l'identifiant de l'entrepôt
                warehouses.append(wid)
                # Calculer la distance de l'entrepôt
                wh_y, wh_x = warehouse_df.iloc[wid,:][['row', 'col']].tolist()
                dist = ((wh_y - self.future_position[1])**2 + (wh_x - self.future_position[0])**2)**0.5
                # Calculer la distance de l'entrepôt
                distance.append(dist)
        
        try:
            # Obtenez l'identifiant de l'entrepôt le plus proche
            minimal_distance = min(distance)
            w_list_id = distance.index(minimal_distance)
            minimal_wh_id = warehouses[w_list_id]

            # Retirer l'article de l'entrepôt le plus proche
            product_df.loc[self.at_product, "warehouse_" + str(minimal_wh_id)] -= 1

            # Calculer la bonne distance
            proper_distance = math.ceil(minimal_distance)

            # Construire l'enregistreur
            logger = (product_df, warehouse_df)

            return proper_distance, warehouse_df.iloc[w_list_id][["col", "row"]].to_list(), logger
        
        except Exception as e:
            
            error_content = {
                "error": str(e),
                "state": self.write_summary(),
                "product": product_df.to_dict(),
                "warehouse": warehouse_df.to_dict()
            }
            
            with open("./log_error.pickle", "wb") as handle:
                pickle.dump(error_content, handle)
            
            sys.exit("Error!!!!")
        
        
    def write_summary(self):
        """
            Méthode pour créer un résumé de l'état du drone pour le débogage.
        """
        return f"""Drone {self.drone_id} - va à l'entrepôt 
                :: order {self.at_order} 
                :: product {self.at_product} 
                :: deliveries {self._delivered_orders}
                :: distance {self._distance}"""
    
    
    def summary(self):
        """
            Méthode qui donne le résumé de l'état du drone pour le débogage.        """
        # Avoir  le résumé du drone
        print(self.write_summary())
    

# Construction du simulateur

## Creation des ordres



In [ ]:
order_list = []
for order in order_df.index.to_list():
    
    # Avoir l'ordre specifique
    num_items = order_df.iloc[order,-3]
    prod_codes = order_df.iloc[order, :num_items].to_list()
    coordinates = order_df.iloc[order, -2:].to_list()
    
    # créer le contenu de l'ordre 
    order_content = (order, prod_codes, coordinates)
    
    # concater les ordres 
    order_list.append(order_content)

## Creation de l'enregistreur des évenements

Creation the logger to control the information of the warehouse products and the warehouses coordinates that will be provided to the drones.

In [ ]:
def build_logger(products_df, warehouse_df):
    """
        Crée la journalisation de la disposition de l’entrepôt, de produits et des informations d’emplacement de l’entrepôt. 
    """
    return (products_df.copy(), warehouse_df.copy()) # Warehouse Products Storage and Warehouse Locations 

## Creation drones

Le but de cette étape est de créer  tous les drones avec un nombre spécifique de ordres à livrer.


In [ ]:
def create_drones(num_drones=None, orders=None, opd=35, verbose=False): 
    
    
    # calculer le nombre de dorne
    n_drones = math.ceil(len(orders) / opd) 
    
    if n_drones > num_drones:
        n_drones = num_drones
    
    drones = []
    for k in range(n_drones):
        ki = opd * k
        kii = opd * (k + 1)
        if kii > len(orders):
            kii = len(orders) 
        drones.append( drone(did=k+1, delivery_order=orders[ki:kii], verbose=verbose) )
    
    return drones

## Creation de simulateur



In [ ]:
def simulate_drones(drones=None, steps=None, logger=None):
    """
        cette fonction va simuler tous les drones un nombre de pas dans le futur, et en retour on va avoir une liste avec tous 
        les drones pour chaque pas dans le futur
        
        :param list drones: liste des drones crées 
        :param int steps: le nombre de pas dans le futur 
        :param tuple logger: l'information intial sur les produits dans les entrepots
        
        :retunrs: liste des drones apres les pas
        :rtype: list
    """
    
    # faire commencer les drones
    for drone in drones:
        logger = drone.start(wh_logger=logger)
        
    # faire des pas pour les drones 
    for step in range(steps):
        for drone in drones:
            result = drone.step(wh_logger=logger)
            if result != None:
                logger = result
    
    return drones

### Tester le simulateur
Nous créons d'abord l'enregistreur pour la simulation... le 'tableau' avec le nombre de produits fourni dans chaque entrepôt.

Deuxièmement, nous créons tous les drones, en fonction de la liste des commandes que nous voulons livrer.

Troisièmement, nous simulons les « pas » des drones dans le futur.

In [ ]:
# Construire l'enregistreur des evenements
logger = build_logger(products_df, warehouse_df)

# Création des drones
drones = create_drones(num_drones=num_drones, orders=order_list, verbose=False)

# Simuler les movements des drones
sim_drones = simulate_drones(drones=drones, steps=num_turns, logger=logger)


## Creation fonction de cout

In [ ]:
def algorithm_cost(orders, product_df, warehouse_df, order_df, num_drones, num_orders_per_drone, num_turns):
    """
        Algorithme qui, basé sur la liste des commandes à livrer, simulera tous les drones et renverra le
        Pourcentage de la distance parcourue par ce drone particulier et le pourcentage de commandes livrées pour tous les
        Drones.
    """
    # avoir les ordres
    order_ids = list(set([int(i) for i in orders]))
    
    order_list = []
    for order in order_ids:
        # avoir le contenu pour chaque drones
        num_items = order_df.iloc[order,-3]
        prod_codes = order_df.iloc[order, :num_items].to_list()
        coordinates = order_df.iloc[order, -2:].to_list()
        # concater les ordres
        order_list.append( (order, prod_codes, coordinates) )
    
    
    # construire l'enregistreur des evenements
    logger = build_logger(products_df, warehouse_df)
    
    # Creer une drone pour chaque liste de commandes
    drones = create_drones(num_drones=num_drones, opd=num_orders_per_drone, orders=order_list, verbose=False)

    # Simuler les mouvements des drones
    sim_drones = simulate_drones(drones=drones, steps=num_turns, logger=logger)  
    
    # calculer le cout: pourcentage de distance et pourcentage des ordres livrés 
    distance = 0
    delivered_orders = 0
    for drone in sim_drones:
        distance += drone._distance / num_turns
        delivered_orders += drone._delivered_orders / len(orders)
#     print(distance)
#     print(len(sim_drones))
    
    distance = 100 * distance / len(sim_drones)
    delivered_orders = 100 * delivered_orders

    # retour le cout à minimiser 
    return distance + (100 - delivered_orders)

# tester le simulatur

In [ ]:
import time

num_parameters = len(order_df)
orders_ = list(range(num_parameters))

time_b = time.time() 
test_cost= algorithm_cost(orders_, products_df, warehouse_df, order_df, num_drones, 35, 10_000)
time_a = time.time() 
print(f"le temps d'exécution est  {round(time_a - time_b, 2)} seconds")

## Optimisation

In [ ]:
 from scipy.optimize import differential_evolution, dual_annealing, shgo, basinhopping,brute

### Differential Evolution

In [ ]:
import time

time_b = time.time() 

# avoir le list de bound
bound_list = [(0, len(order_df))] * 15


arguments = (products_df,  #  
             warehouse_df, # 
             order_df,     # 
             num_drones,   # 
             15,           # livraison par drone
             10_000)        #  turns/steps


summary_DE = differential_evolution(algorithm_cost,
                                 bound_list,
                                 args=arguments,
                                 maxiter=500,
                                 tol=0.05,
                                 recombination=1,
                                    mutation = 1.99,
                                 #popsize=35,
                                 #updating='deferred',
                                 #workers=-1,
                                 disp=True
                                )

time_a = time.time() 

In [2]:
print("La méthode de génitique évolutionnaire  nous donne un cout de :126.66666666666667")#,  summary_DE.fun)
print("Le nombre d'itération est de : 19 ")#, summary_DE.nit)
# print("Nombre d'évaluations de la fonction de cout et de son Jacobien et Hessien : ", summary_DE.nfev)

La méthode de génitique évolutionnaire  nous donne un cout de :126.66666666666667
Le nombre d'itération est de : 19 


In [ ]:
print(f"Temps de convergence {time_a - time_b} seconds.")

In [ ]:
drone_orders = list(set([int(i) for i in summary_DE.x]))

drone_orders_df = order_df.iloc[drone_orders]

x_coordinates = drone_orders_df["coor_x"].to_list()
y_coordinates = drone_orders_df["coor_y"].to_list()

import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=order_df["coor_x"],
    y=order_df["coor_y"],
    name="Orders",
    mode="markers",
    marker_symbol="circle",
    marker_size=3, 
))

fig.add_trace(go.Scatter(
    x=warehouse_df["col"],
    y=warehouse_df["row"],
    name="Warehouses",
    mode="markers",
    marker_symbol="hexagram",
    marker_size=10, 
))

fig.add_trace(go.Scatter(
    x=[0] + x_coordinates, 
    y=[0] + y_coordinates, 
    name="Deliveries", 
    mode="lines+markers", 
    marker_size=15
))

fig.add_trace(go.Scatter(
    x=[x_coordinates[0]],
    y=[y_coordinates[0]],
    name="First Delivery",
    mode="markers",
    marker_symbol="x",
    marker_size=15
))
    
fig.add_trace(go.Scatter(
    x=[x_coordinates[-1]],
    y=[y_coordinates[-1]],
    name="Last Delivery",
    mode="markers",
    marker_symbol="x",
    marker_size=15, 
))
    
fig.add_trace(go.Scatter(
    x=[0], y=[0],
    name="Starting position",
    mode="markers",
    marker_symbol="star-square",
    marker_size=15, 
))
    
fig.show()

### Recuit simulé

In [ ]:
time_b = time.time() 

bound_list = [(0, len(order_df))] * 15


arguments = (products_df, 
             warehouse_df, 
             order_df,     
             num_drones,
             15,        
             10_000)    


summary_SA = dual_annealing(algorithm_cost,
                         bound_list,
                         args=arguments,
                         maxiter=500)

time_a = time.time() 

In [ ]:
print("la méthode de recuit simué nous donne un cout de :",  summary_SA.fun)


print("Le nombre d'itération est de : ", summary_SA.nit)
print("Nombre d'évaluations de la fonction de cout et de son Jacobien et Hessien : ", summary_SA.nfev)

In [ ]:
print(f"Le temps de convergence {time_a - time_b} secondes.")

In [ ]:
drone_orders = list(set([int(i) for i in summary_SA.x]))

drone_orders_df = order_df.iloc[drone_orders]

x_coordinates = drone_orders_df["coor_x"].to_list()
y_coordinates = drone_orders_df["coor_y"].to_list()

import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=order_df["coor_x"],
    y=order_df["coor_y"],
    name="Orders",
    mode="markers",
    marker_symbol="circle",
    marker_size=5, 
))

fig.add_trace(go.Scatter(
    x=warehouse_df["col"],
    y=warehouse_df["row"],
    name="Warehouses",
    mode="markers",
    marker_symbol="hexagram",
    marker_size=20, 
))

fig.add_trace(go.Scatter(
    x=[0] + x_coordinates, 
    y=[0] + y_coordinates, 
    name="Deliveries", 
    mode="lines+markers", 
    marker_size=10
))

fig.add_trace(go.Scatter(
    x=[x_coordinates[0]],
    y=[y_coordinates[0]],
    name="First Delivery",
    mode="markers",
    marker_symbol="x",
    marker_size=15, 
))
    
fig.add_trace(go.Scatter(
    x=[x_coordinates[-1]],
    y=[y_coordinates[-1]],
    name="Last Delivery",
    mode="markers",
    marker_symbol="x",
    marker_size=15, 
))
    
fig.add_trace(go.Scatter(
    x=[0], y=[0],
    name="Starting position",
    mode="markers",
    marker_symbol="star-square",
    marker_size=15, 
))
    
fig.show()